In [3]:
USER_AGENT = 'Channel1Scraper:1.0 (by u/AlarmingSignal4018)'

In [6]:
# pull_stories.py

import praw
import os
from dotenv import load_dotenv, find_dotenv


load_dotenv(find_dotenv(usecwd=True), override=True)

client_secret = os.getenv("CLIENT_SECRET")
user_id = os.getenv("USER_ID")

# Connect to Reddit
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent= USER_AGENT
)

def fetch_trending(subreddit_name="all", limit=10):
    """Fetch trending stories from a subreddit (default: all)"""
    subreddit = reddit.subreddit(subreddit_name)
    stories = []

    print(f"\n🔥 Top {limit} rising posts in r/{subreddit_name} 🔥")
    for post in subreddit.rising(limit=limit):
        story = {
            "title": post.title,
            "score": post.score,
            "comments": post.num_comments,
            "url": post.url,
            "text": post.selftext[:500]  # preview first 500 chars
        }
        stories.append(story)
        print(f"{story['title']} | 👍 {story['score']} | 💬 {story['comments']}")

    return stories


if __name__ == "__main__":
    trending = fetch_trending(limit=20)

    # Save results to JSON
    import json
    with open("trending.json", "w", encoding="utf-8") as f:
        json.dump(trending, f, indent=2, ensure_ascii=False)

    print("\n✅ Saved trending stories to trending.json")



🔥 Top 20 rising posts in r/all 🔥
Maine/everyone deserves better than Susan Collins | 👍 839 | 💬 13
Democratic candidate running for U.S. Senate in Maine, Graham Platner: "No one cares that you pretend to be remorseful as you sell out to lobbyists. Symbolic opposition does not reopen hospitals. Weak condemnations do not bring back Roe V Wade. Maine deserves better than Susan Collins" | 👍 870 | 💬 15
Aaaany second now... | 👍 349 | 💬 22
So very Trumpian | 👍 691 | 💬 25
The safety of a rally car | 👍 1869 | 💬 98
Zohran Mamdani responds to Islamophobic trolling in the best way | 👍 1859 | 💬 67
Aquatic plant producing oxygen | 👍 1007 | 💬 28
Adulthood in One Sentence | 👍 187 | 💬 3
My mom found somebody's tooth implant in her quesadilla | 👍 253 | 💬 53
Meow~ (@JizokuArt) | 👍 438 | 💬 35
Democratic Rep. Jerry Nadler to retire from Congress | 👍 310 | 💬 36
Stop falling for "luxury" apartments. | 👍 150 | 💬 44
Bike theft gone wrong | 👍 230 | 💬 39
Dafne Keen clink | 👍 128 | 💬 1
Future nominee Dwayne Johns

In [7]:
import praw
from dotenv import load_dotenv, find_dotenv


load_dotenv(find_dotenv(usecwd=True), override=True)

client_secret = os.getenv("CLIENT_SECRET")
user_id = os.getenv("USER_ID")

# Connect to Reddit
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent= USER_AGENT
)

def fetch_long_stories(subreddit_name="AskReddit", limit=20, min_score=5000, min_length=500):
    """
    Pull long, high-engagement posts from a subreddit.
    
    Args:
        subreddit_name: which subreddit to pull from
        limit: how many posts to scan
        min_score: minimum upvotes required
        min_length: minimum length of the body text (characters)
    """
    subreddit = reddit.subreddit(subreddit_name)
    stories = []

    print(f"\n📚 Long high-engagement posts from r/{subreddit_name}:")
    for post in subreddit.top(time_filter="week", limit=limit):  # top posts of the week
        if post.score >= min_score and len(post.selftext) >= min_length:
            story = {
                "title": post.title,
                "score": post.score,
                "comments": post.num_comments,
                "url": "https://reddit.com" + post.permalink,
                "text": post.selftext
            }
            stories.append(story)
            print(f"\nTitle: {story['title']}")
            print(f"👍 {story['score']} | 💬 {story['comments']}")
            print(f"🔗 {story['url']}")
            print(f"📝 Preview:\n{story['text'][:400]}...\n")

    return stories


if __name__ == "__main__":
    long_posts = fetch_long_stories(limit=50, min_score=0, min_length=0)

    # Save to file
    import json
    with open("long_stories.json", "w", encoding="utf-8") as f:
        json.dump(long_posts, f, indent=2, ensure_ascii=False)

    print("\n✅ Saved long stories to long_stories.json")



📚 Long high-engagement posts from r/AskReddit:

Title: Trump said today: “I have the right to do anything I want to do. I’m the president of the United States”. People from the US, what are your thoughts on this?
👍 57263 | 💬 13705
🔗 https://reddit.com/r/AskReddit/comments/1n0xxaf/trump_said_today_i_have_the_right_to_do_anything/
📝 Preview:
...


Title: What is a job that pays extremely well but no one realizes it?
👍 13166 | 💬 6077
🔗 https://reddit.com/r/AskReddit/comments/1n4g11a/what_is_a_job_that_pays_extremely_well_but_no_one/
📝 Preview:
...


Title: To those who have left the MAGA movement, what was your wake-up call?
👍 11190 | 💬 6863
🔗 https://reddit.com/r/AskReddit/comments/1n14pol/to_those_who_have_left_the_maga_movement_what_was/
📝 Preview:
...


Title: What would the GOP power struggle look like if Donald Trump dies?
👍 8667 | 💬 2955
🔗 https://reddit.com/r/AskReddit/comments/1n40xp2/what_would_the_gop_power_struggle_look_like_if/
📝 Preview:
...


Title: What Do You Think Would

In [ ]:
# pull_best_comments.py
import time
import json
import re
from typing import List, Dict, Any
import praw
from prawcore.exceptions import RequestException, ResponseException, ServerError

# ----------------------------
# SETTINGS
# ----------------------------
DRAMA_SUBREDDITS = [
    "AmItheAsshole", "BestofRedditorUpdates", "relationship_advice",
    "TrueOffMyChest", "relationships", "JustNoMIL", "entitledparents",
    "OffMyChest", "survivinginfidelity"
]
# You can also include 'all' in the list, or just target "AskReddit" specifically

DRAMA_KEYWORDS = [
    "divorce","cheat","affair","wedding","engagement","pregnant","custody",
    "family","mother-in-law","entitled","betray","gaslight","toxic","lawsuit",
    "will","inherit","ex","backfire","secret","lied","scammed","abusive"
]
DRAMA_RE = re.compile(r"\b(" + "|".join(re.escape(k) for k in DRAMA_KEYWORDS) + r")\b", re.I)

# Post filters
MIN_POST_SCORE = 2000          # baseline quality gate
MIN_POST_COMMENTS = 150
POSTS_PER_SUBREDDIT = 10       # how many candidate posts to scan per subreddit
POST_TIME_FILTER = "week"      # 'day' | 'week' | 'month' | 'year' | 'all'

# Comment filters
MIN_COMMENT_SCORE = 200
MIN_COMMENT_LENGTH = 400       # characters
MAX_COMMENTS_TO_SCAN = 600     # safety cap per post to limit API work
TOP_COMMENTS_PER_POST = 8      # how many to keep per post

# Output
OUTFILE = "best_comments.json"

# ----------------------------
# REDDIT CLIENT
# ----------------------------
from dotenv import load_dotenv, find_dotenv


load_dotenv(find_dotenv(usecwd=True), override=True)

client_secret = os.getenv("CLIENT_SECRET")
user_id = os.getenv("USER_ID")

# Connect to Reddit
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent= USER_AGENT
)

# ----------------------------
# HELPERS
# ----------------------------
def looks_removed(text: str) -> bool:
    t = (text or "").strip().lower()
    return t in ("[removed]", "[deleted]")

def comment_engagement_score(c) -> float:
    """
    A simple, tunable scoring function.
    We reward: upvotes, awards, replies, controversy, length, and drama keywords.
    """
    score = getattr(c, "score", 0) or 0
    awards = getattr(c, "total_awards_received", 0) or 0
    controversiality = getattr(c, "controversiality", 0) or 0
    body = getattr(c, "body", "") or ""
    length = len(body)

    # Direct replies count (only loaded if tree expanded; still a decent proxy)
    try:
        direct_replies = len(c.replies)
    except Exception:
        direct_replies = 0

    # Base: votes + awards + replies
    s = score + 50 * awards + 4 * direct_replies

    # Heated debates get a bump
    s += 120 * controversiality

    # Reward long-form (cap the boost)
    s += min(length // 300, 6) * 25  # +25 per 300 chars up to 6 chunks

    # Drama keywords bonus
    if DRAMA_RE.search(body):
        s += 120

    # OP comments are often interesting but not always ideal for “commentary” videos.
    # If you want to favor OP’s clarifications, uncomment:
    # if getattr(c, "is_submitter", False):
    #     s += 40

    return float(s)

def fetch_candidate_submissions(subreddit_name: str) -> List[Any]:
    """
    Pull high-engagement posts to mine for comments.
    We use .top(time_filter=POST_TIME_FILTER) to bias toward proven engagement.
    """
    sub = reddit.subreddit(subreddit_name)
    candidates = []
    # You can mix sources; here we use top-week. You can also try rising() for “breaking.”
    for post in sub.top(time_filter=POST_TIME_FILTER, limit=POSTS_PER_SUBREDDIT * 2):
        if post.score >= MIN_POST_SCORE and post.num_comments >= MIN_POST_COMMENTS and not post.stickied:
            candidates.append(post)
            if len(candidates) >= POSTS_PER_SUBREDDIT:
                break
    return candidates

def fetch_best_comments_for_submission(submission) -> List[Dict[str, Any]]:
    """
    Expand comment tree, filter, score, and return the best comments.
    """
    # Ensure best/top order
    submission.comment_sort = "top"
    submission.comments.replace_more(limit=0)  # expand all "MoreComments"

    collected = []
    count_scanned = 0

    # Flattened traversal (ordered by current sort)
    for c in submission.comments.list():
        if count_scanned >= MAX_COMMENTS_TO_SCAN:
            break
        count_scanned += 1

        # Skip non-Comment objects defensively
        if c.__class__.__name__ != "Comment":
            continue

        body = getattr(c, "body", "") or ""
        if looks_removed(body):
            continue
        if len(body) < MIN_COMMENT_LENGTH:
            continue
        if getattr(c, "score", 0) < MIN_COMMENT_SCORE:
            continue
        if getattr(c, "stickied", False):
            continue  # skip auto-mod stickies

        s = comment_engagement_score(c)

        collected.append({
            "submission_id": submission.id,
            "submission_title": submission.title,
            "submission_url": "https://reddit.com" + submission.permalink,
            "submission_score": submission.score,
            "submission_num_comments": submission.num_comments,

            "comment_id": c.id,
            "comment_permalink": "https://reddit.com" + c.permalink,
            "author": str(getattr(c, "author", None)) if getattr(c, "author", None) else "[deleted]",
            "score": c.score,
            "awards": getattr(c, "total_awards_received", 0) or 0,
            "controversiality": getattr(c, "controversiality", 0) or 0,
            "is_submitter": getattr(c, "is_submitter", False),
            "direct_replies": len(c.replies) if hasattr(c, "replies") else 0,
            "length": len(body),
            "body": body,
            "engagement_score": s
        })

    # Sort and take top N
    collected.sort(key=lambda x: x["engagement_score"], reverse=True)
    return collected[:TOP_COMMENTS_PER_POST]

def run(subreddits: List[str]) -> List[Dict[str, Any]]:
    results = []
    for name in subreddits:
        print(f"\n=== Mining r/{name} ===")
        try:
            posts = fetch_candidate_submissions(name)
        except (RequestException, ResponseException, ServerError) as e:
            print(f"Skipping r/{name} due to API error: {e}")
            continue

        for i, post in enumerate(posts, 1):
            print(f"  [{i}/{len(posts)}] {post.title[:90]}...  (👍 {post.score} | 💬 {post.num_comments})")
            try:
                top_comments = fetch_best_comments_for_submission(post)
                results.extend(top_comments)
                time.sleep(1.2)  # be gentle to rate limits
            except (RequestException, ResponseException, ServerError) as e:
                print(f"    Failed on comments for {post.id}: {e}")
                time.sleep(1.5)

        # short pause between subreddits
        time.sleep(1.5)

    # Global sort to surface absolute bangers
    results.sort(key=lambda x: x["engagement_score"], reverse=True)
    return results

if __name__ == "__main__":
    # Example 1: specifically r/AskReddit
    # target = ["AskReddit"]

    # Example 2: drama-heavy mix
    target = DRAMA_SUBREDDITS + ["AskReddit"]

    best = run(target)

    # Save
    with open(OUTFILE, "w", encoding="utf-8") as f:
        json.dump(best, f, indent=2, ensure_ascii=False)

    print(f"\n✅ Saved {len(best)} high-engagement, long comments → {OUTFILE}")



=== Mining r/AmItheAsshole ===
  [1/10] AITA - Do not want a service dog to participate in my wedding....  (👍 10409 | 💬 9225)


KeyboardInterrupt: 

In [ ]:
# pull_best_comments.py
import time
import json
import re
from typing import List, Dict, Any
import praw
from prawcore.exceptions import RequestException, ResponseException, ServerError, Forbidden

# ----------------------------
# SETTINGS
# ----------------------------
DRAMA_SUBREDDITS = [
    "AmItheAsshole", "BestofRedditorUpdates", "relationship_advice",
    "TrueOffMyChest", "relationships", "JustNoMIL", "entitledparents",
    "OffMyChest", "survivinginfidelity"
]
# You can also include 'all' in the list, or just target "AskReddit" specifically

DRAMA_KEYWORDS = [
    "divorce","cheat","affair","wedding","engagement","pregnant","custody",
    "family","mother-in-law","entitled","betray","gaslight","toxic","lawsuit",
    "will","inherit","ex","backfire","secret","lied","scammed","abusive","update","boundary"
]
DRAMA_RE = re.compile(r"\b(" + "|".join(re.escape(k) for k in DRAMA_KEYWORDS) + r")\b", re.I)

# Post filters (for **posts** we’ll also use these gates)
MIN_POST_SCORE = 2000          # baseline quality gate
MIN_POST_COMMENTS = 150
POSTS_PER_SUBREDDIT = 10       # how many candidate posts to scan per subreddit
POST_TIME_FILTER = "week"      # 'day' | 'week' | 'month' | 'year' | 'all'

# Extra post gate for long-form stories
MIN_POST_LENGTH = 700          # chars in selftext (longer = more likely a full story)

# Comment filters
MIN_COMMENT_SCORE = 200
MIN_COMMENT_LENGTH = 400       # characters
MAX_COMMENTS_TO_SCAN = 600     # safety cap per post to limit API work
TOP_COMMENTS_PER_POST = 8      # how many to keep per post

# Output
OUTFILE = "best_comments.json"             # (same name as your original)
POSTS_OUTFILE = "best_posts.json"          # new: long story posts (title + selftext)
COMBINED_OUTFILE = "stories_with_comments.json"  # new: post + its top comments

# ----------------------------
# REDDIT CLIENT
# (uses your existing globals: CLIENT_ID / CLIENT_SECRET / USER_AGENT)
# ----------------------------
from dotenv import load_dotenv, find_dotenv


load_dotenv(find_dotenv(usecwd=True), override=True)

client_secret = os.getenv("CLIENT_SECRET")
user_id = os.getenv("USER_ID")

# Connect to Reddit
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent= USER_AGENT
)

# ----------------------------
# HELPERS
# ----------------------------
def looks_removed(text: str) -> bool:
    t = (text or "").strip().lower()
    return t in ("[removed]", "[deleted]")

def comment_engagement_score(c) -> float:
    """
    A simple, tunable scoring function.
    We reward: upvotes, awards, replies, controversy, length, and drama keywords.
    """
    score = getattr(c, "score", 0) or 0
    awards = getattr(c, "total_awards_received", 0) or 0
    controversiality = getattr(c, "controversiality", 0) or 0
    body = getattr(c, "body", "") or ""
    length = len(body)

    # Direct replies count (only loaded if tree expanded; still a decent proxy)
    try:
        direct_replies = len(c.replies)
    except Exception:
        direct_replies = 0

    # Base: votes + awards + replies
    s = score + 50 * awards + 4 * direct_replies

    # Heated debates get a bump
    s += 120 * controversiality

    # Reward long-form (cap the boost)
    s += min(length // 300, 6) * 25  # +25 per 300 chars up to 6 chunks

    # Drama keywords bonus
    if DRAMA_RE.search(body):
        s += 120

    # OP comments are often interesting but not always ideal for “commentary” videos.
    # If you want to favor OP’s clarifications, uncomment:
    # if getattr(c, "is_submitter", False):
    #     s += 40

    return float(s)

# ----------------------------
# POSTS: discovery (long story posts)
# ----------------------------
def fetch_long_story_posts(subreddit_name: str) -> List[Dict[str, Any]]:
    """
    Discover long, engaged **posts** (selftext stories) from a subreddit.
    Uses .top() for the configured POST_TIME_FILTER and applies your post gates.
    """
    sub = reddit.subreddit(subreddit_name)
    results = []

    print(f"  > Scanning posts from r/{subreddit_name} (top/{POST_TIME_FILTER}) …")
    # Pull a bit wider than POSTS_PER_SUBREDDIT so filters can cut down to ~target
    limit = max(POSTS_PER_SUBREDDIT * 5, 30)
    for post in sub.top(time_filter=POST_TIME_FILTER, limit=limit):
        try:
            if post.stickied:
                continue
            if not getattr(post, "is_self", False):
                continue

            text = (post.selftext or "").strip()
            if not text or looks_removed(text):
                continue
            if len(text) < MIN_POST_LENGTH:
                continue
            if post.score < MIN_POST_SCORE or post.num_comments < MIN_POST_COMMENTS:
                continue

            row = {
                "id": post.id,
                "subreddit": str(post.subreddit),
                "title": post.title,
                "url": "https://reddit.com" + post.permalink,
                "score": int(post.score),
                "num_comments": int(post.num_comments),
                "upvote_ratio": getattr(post, "upvote_ratio", None),
                "created_utc": float(post.created_utc),
                "length": len(text),
                "selftext": text
            }
            results.append(row)

            if len(results) >= POSTS_PER_SUBREDDIT:
                break
        except Exception:
            # Skip weird/unexpected objects
            continue

    # Rank posts very simply: score + 2*comments (+ keyword bump)
    def post_rank_key(p):
        base = p["score"] + 2 * p["num_comments"]
        if DRAMA_RE.search(p["title"]) or DRAMA_RE.search(p["selftext"]):
            base += 200
        return base

    results.sort(key=post_rank_key, reverse=True)
    print(f"  > Found {len(results)} long story posts in r/{subreddit_name}")
    return results

# ----------------------------
# COMMENTS: mining (top + controversial)
# ----------------------------
def fetch_best_comments_for_submission(submission) -> List[Dict[str, Any]]:
    """
    Expand comment tree, filter, score, and return the best comments.
    We scan **both** 'top' and 'controversial' sorts, with long pauses.
    """
    collected = []
    seen_ids = set()
    for sort in ("top", "controversial"):
        try:
            submission.comment_sort = sort
            submission.comments.replace_more(limit=0)  # expand all "MoreComments"
        except (RequestException, ResponseException, ServerError, Forbidden):
            continue

        count_scanned = 0
        for c in submission.comments.list():
            if count_scanned >= MAX_COMMENTS_TO_SCAN:
                break
            count_scanned += 1

            if c.__class__.__name__ != "Comment":
                continue
            if c.id in seen_ids:
                continue

            body = getattr(c, "body", "") or ""
            if looks_removed(body):
                continue
            if len(body) < MIN_COMMENT_LENGTH:
                continue
            if getattr(c, "score", 0) < MIN_COMMENT_SCORE:
                continue
            if getattr(c, "stickied", False):
                continue  # skip auto-mod stickies

            s = comment_engagement_score(c)
            seen_ids.add(c.id)
            collected.append({
                "submission_id": submission.id,
                "submission_title": submission.title,
                "submission_url": "https://reddit.com" + submission.permalink,
                "submission_score": submission.score,
                "submission_num_comments": submission.num_comments,

                "comment_id": c.id,
                "comment_permalink": "https://reddit.com" + c.permalink,
                "author": str(getattr(c, "author", None)) if getattr(c, "author", None) else "[deleted]",
                "score": int(getattr(c, "score", 0) or 0),
                "awards": int(getattr(c, "total_awards_received", 0) or 0),
                "controversiality": int(getattr(c, "controversiality", 0) or 0),
                "is_submitter": bool(getattr(c, "is_submitter", False)),
                "direct_replies": int(len(c.replies) if hasattr(c, "replies") else 0),
                "length": len(body),
                "body": body,
                "engagement_score": s,
                "sort_source": sort
            })

        # ~3s rest between sorts
        time.sleep(3.0)

    # Sort and take top N
    collected.sort(key=lambda x: x["engagement_score"], reverse=True)
    return collected[:TOP_COMMENTS_PER_POST]

# ----------------------------
# Orchestrator (keeps same signature)
# ----------------------------
def run(subreddits: List[str]) -> List[Dict[str, Any]]:
    """
    EXACT same call shape as your original: returns a flat list of top comments
    and (side effect) also writes posts and combined files.
    """
    all_posts = []
    all_comments = []
    combined = []

    for name in subreddits:
        print(f"\n=== Mining r/{name} ===")

        # 1) Pull **posts** (long-form stories)
        try:
            posts = fetch_long_story_posts(name)
            all_posts.extend(posts)
        except (RequestException, ResponseException, ServerError, Forbidden) as e:
            print(f"Skipping r/{name} due to API error (posts): {e}")
            # 3s rest even on error
            time.sleep(3.0)
            continue

        # 2) For each post, pull **comments**
        for i, post in enumerate(posts, 1):
            print(f"  [{i}/{len(posts)}] {post['title'][:90]}...  (👍 {post['score']} | 💬 {post['num_comments']})")
            try:
                subm = reddit.submission(id=post["id"])
                top_comments = fetch_best_comments_for_submission(subm)
                all_comments.extend(top_comments)
                combined.append({
                    "post": post,
                    "top_comments": top_comments
                })
            except (RequestException, ResponseException, ServerError, Forbidden) as e:
                print(f"    Failed on comments for {post['id']}: {e}")

            # ~3s rest between submissions
            time.sleep(3.0)

        # ~3s rest between subreddits
        time.sleep(3.0)

    # Sort outputs
    all_comments.sort(key=lambda x: x["engagement_score"], reverse=True)

    # Persist extra files here (side effects)
    try:
        with open(POSTS_OUTFILE, "w", encoding="utf-8") as f:
            json.dump(all_posts, f, indent=2, ensure_ascii=False)
        print(f"\n✅ Saved {len(all_posts)} long story posts → {POSTS_OUTFILE}")

        with open(COMBINED_OUTFILE, "w", encoding="utf-8") as f:
            json.dump(combined, f, indent=2, ensure_ascii=False)
        print(f"✅ Saved {len(combined)} combined records (post + comments) → {COMBINED_OUTFILE}")
    except Exception as e:
        print(f"⚠️ Failed writing posts/combined files: {e}")

    # Return comments (so your main keeps working identically)
    return all_comments

# ----------------------------
# MAIN (unchanged calling style)
# ----------------------------
if __name__ == "__main__":
    # Example 1: specifically r/AskReddit
    # target = ["AskReddit"]

    # Example 2: drama-heavy mix
    target = DRAMA_SUBREDDITS + ["AskReddit"]

    best = run(target)

    # Save (same as your original)
    with open(OUTFILE, "w", encoding="utf-8") as f:
        json.dump(best, f, indent=2, ensure_ascii=False)

    print(f"\n✅ Saved {len(best)} high-engagement, long comments → {OUTFILE}")



=== Mining r/AmItheAsshole ===
  > Scanning posts from r/AmItheAsshole (top/week) …
  > Found 10 long story posts in r/AmItheAsshole
  [1/10] AITA - Do not want a service dog to participate in my wedding....  (👍 10612 | 💬 9347)


/var/folders/n2/rstg0c3j7hv2shgc8szmsj2r0000gp/T/ipykernel_60185/2257734352.py:181: UserWarning: The comments for this submission have already been fetched, so the updated comment_sort will not have any effect.
  submission.comment_sort = sort


  [2/10] AITA for not changing my annual backyard party plans for my boyfriend and his kids?...  (👍 9298 | 💬 1206)
  [3/10] AITA for telling my sister she wasn't always the chosen one?...  (👍 8095 | 💬 1699)
  [4/10] AITA Tension After My Father Passed Away and I had to Leave 36 Hours Later Because of My I...  (👍 7046 | 💬 1548)
  [5/10] AITA for asking my girlfriend to tell me what flavour of smoothie she wanted?...  (👍 6081 | 💬 687)
